In [125]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key


In [126]:
cities_data = "cities.csv"
cities_df = pd.read_csv(cities_data)
cities_df

City  Cloudiness Country        Date  Humidity    Lat     Lng  \
0         Bridgton          90      US  1578203306        86  44.05  -70.71   
1          Qaanaaq           0      GL  1578203308        51  77.48  -69.36   
2         Ishigaki          40      JP  1578203309        69  24.34  124.16   
3            Kapaa          90      US  1578203077        88  22.08 -159.32   
4           Itoman          75      JP  1578203312        56  26.12  127.67   
..             ...         ...     ...         ...       ...    ...     ...   
528        Talnakh          90      RU  1578203922        84  69.49   88.39   
529       Kochubey         100      RU  1578203923        86  44.39   46.54   
530       Cururupu         100      BR  1578203925        98  -1.82  -44.87   
531          Yatou          40      CM  1578203926        94   3.63    9.81   
532  Villa Sandino           0      NI  1578203927        92  12.05  -84.99   

     Max Temp  Wind Speed  
0       34.00        8.05  
1      -39.08        5.97  
2       77.00        9.17  
3       75.20       20.80  
4       69.80        8.05  
..        ...         ...  
528      8.60       35.79  
529     41.14       12.93  
530     73.00        1.68  
531     77.00        1.12  
532     68.34        9.75  

[533 rows x 9 columns]

In [127]:
gmaps.configure(api_key=g_key)

In [128]:
locations = cities_df[['Lat', 'Lng']]
weights = cities_df['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, dissipating=False))
fig


Figure(layout=FigureLayout(height='420px'))

In [129]:
wind_df = cities_df.loc[cities_df['Wind Speed'] < 10]
high_temp_df = wind_df[wind_df['Max Temp'] < 80]
low_temp_df = high_temp_df[high_temp_df['Max Temp'] > 70]
hotel_df = low_temp_df[low_temp_df['Cloudiness'] == 0]
hotel_df

City  Cloudiness Country        Date  Humidity    Lat    Lng  \
17   Pozo Colorado           0      PY  1578203326        71 -23.49 -58.80   
113  Cockburn Town           0      TC  1578203378        68  21.46 -71.14   
171        Guhagar           0      IN  1578203505        55  17.48  73.19   
183          Birur           0      IN  1578203519        49  13.62  75.97   
194       La Plata           0      AR  1578203532        88 -34.92 -57.95   
215   San Patricio           0      PY  1578203557        68 -26.98 -56.83   
303    Robertsport           0      LR  1578203660        73   6.75 -11.37   
437        Umarkot           0      PK  1578203815        27  25.37  69.74   

     Max Temp  Wind Speed  
17      74.44        3.67  
113     77.20        7.29  
171     76.50        1.72  
183     78.44        5.99  
194     73.99        2.24  
215     70.97        4.65  
303     78.73        8.12  
437     73.87        6.04

In [130]:
hotel_df['Hotel Name'] = ""
hotel_df.head(10)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


City  Cloudiness Country        Date  Humidity    Lat    Lng  \
17   Pozo Colorado           0      PY  1578203326        71 -23.49 -58.80   
113  Cockburn Town           0      TC  1578203378        68  21.46 -71.14   
171        Guhagar           0      IN  1578203505        55  17.48  73.19   
183          Birur           0      IN  1578203519        49  13.62  75.97   
194       La Plata           0      AR  1578203532        88 -34.92 -57.95   
215   San Patricio           0      PY  1578203557        68 -26.98 -56.83   
303    Robertsport           0      LR  1578203660        73   6.75 -11.37   
437        Umarkot           0      PK  1578203815        27  25.37  69.74   

     Max Temp  Wind Speed Hotel Name  
17      74.44        3.67             
113     77.20        7.29             
171     76.50        1.72             
183     78.44        5.99             
194     73.99        2.24             
215     70.97        4.65             
303     78.73        8.12             
437     73.87        6.04

In [139]:
# target_coordinates = "-23.49, -58.80"
# target_radius = 5000
# target_type = "lodging"

# # set up a parameters dictionary
# params = {
#     "location": target_coordinates,
#     "radius": target_radius,
#     "type": target_type,
#     "key": g_key
# }

# # base url
# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# # run a request using our params dictionary
# response = requests.get(base_url, params=params)
# places_data = response.json()
# print(places_data["results"][0]["name"])

In [144]:
params = {
    "radius": 5000,   
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # f-string to capture each of the city locations
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_name = requests.get(base_url, params=params)
    hotel_name = hotel_name.json()
    
    hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]

narrowed_city_df = hotel_df
narrowed_city_df

City  Cloudiness Country        Date  Humidity    Lat    Lng  \
17   Pozo Colorado           0      PY  1578203326        71 -23.49 -58.80   
113  Cockburn Town           0      TC  1578203378        68  21.46 -71.14   
171        Guhagar           0      IN  1578203505        55  17.48  73.19   
183          Birur           0      IN  1578203519        49  13.62  75.97   
194       La Plata           0      AR  1578203532        88 -34.92 -57.95   
215   San Patricio           0      PY  1578203557        68 -26.98 -56.83   
303    Robertsport           0      LR  1578203660        73   6.75 -11.37   
437        Umarkot           0      PK  1578203815        27  25.37  69.74   

     Max Temp  Wind Speed            Hotel Name  
17      74.44        3.67  Hospedaje la Amistad  
113     77.20        7.29    Osprey Beach Hotel  
171     76.50        1.72        Hotel Seawinds  
183     78.44        5.99     Raheem Sab Layout  
194     73.99        2.24      Hotel Corregidor  
215     70.97        4.65    Granja "La Blanca"  
303     78.73        8.12      Kwepunha Retreat  
437     73.87        6.04  Ciruit House Umerkot

In [145]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [148]:
# From what Eli posted in the Slack channel

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))